#### Cette 1ère étape consiste à scraper les articles trouvés sur le site lemonde.fr qui parlent des sujets abordés dans les discours des gouverneurs de la Banque de France. Pour cela, nous avons choisi de scraper les articles qui correspondent à notre liste des mots clés à rechercher. La liste contient les noms des gouverneurs de la BF durant la période de 2007-2022

## 1/ Importer les librairies nécessaires :

In [ ]:
#Selenium
#!pip install selenium
#!pip install webdriver_manager

from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from selenium.webdriver.support.select import Select 
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome

import warnings
warnings.filterwarnings("ignore")

#BeautifulSoup

import requests
from bs4 import BeautifulSoup

#nltk

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import os
import pandas as pd
import numpy as np
import math
import time
import re

## 2/ Scraping des articles liés à la Banque de France :

In [ ]:
# Ouverture du navigateur et acceptation des cookies :

url = 'https://www.lemonde.fr/recherche/?search_keywords=François%20Villeroy%20De%20Galhau&start_at=01/01/2007&end_at=15/04/2023&search_sort=relevance_desc&page=1'

chromeOptions = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chromeOptions)
driver.get(url)



accept_cookies =  driver.find_element(By.XPATH, "//button[contains(@class, 'gdpr-lmd-button gdpr-lmd-button--big gdpr-lmd-button--main')]")
driver.execute_script("arguments[0].click();", accept_cookies);

time.sleep(5)

#rafraichir la page :
driver.refresh()

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:03<00:00, 1.99MB/s]


In [ ]:
# Liste des gouverneurs de la Banque de France :
liste_keywords_bf = ['françois villeroy de galhau','christian noyer', 'denis beau', 'claude trichet','sylvie goulard']

# Liste du ministère des finances Français :
liste_keywords_mf = ["Bruno Le Maire" ,"Hervé Novelli","Pierre Lellouche","Eric Besson","Christine Lagarde",
                     "Emmanuel Macron", "Pierre Moscovici","Christian Eckert","Frédéric Lefebvre","Christian Estrosi",
                     "Michel Sapin","Martine Pinville","Carole Delga","Valérie Pécresse", "Eric Woerth"]

In [ ]:
#Déterminer le nombre de pages pour chaque url de la liste des gouverneurs de la BF

nombre_pages_bf = []

for keyword in liste_keywords_bf:
    texte  = "%20".join(keyword.split())
    driver.get('https://www.lemonde.fr/recherche/?search_keywords={}&start_at=01/01/2007&end_at=31/12/2022&search_sort=relevance_desc&page=1'.format(texte))
    last_page = int(driver.find_elements(By.CLASS_NAME,'river__pagination')[-1].text)
    nombre_pages_bf.append(last_page)

In [ ]:
nombre_pages_bf

[6, 12, 40, 34, 7]

In [ ]:
#Récupération de tout les liens de la MF de chaque mot clé :

nombre_pages_mf = []

for keyword in liste_keywords_mf:
    texte  = "%20".join(keyword.lower().split())
    driver.get('https://www.lemonde.fr/recherche/?search_keywords={}&start_at=01/01/2007&end_at=31/12/2022&search_sort=relevance_desc&page=1'.format(texte))
    last_page = int(driver.find_elements(By.CLASS_NAME,'river__pagination')[-1].text)
    nombre_pages_mf.append(last_page)

In [ ]:
nombre_pages_mf

[164, 10, 16, 48, 103, 250, 71, 13, 23, 49, 58, 2, 10, 108, 73]

In [ ]:
#Récupération de tout les liens de la BF de chaque mot clé :

liste_liens_bf = []

for (keyword, last_page) in zip(liste_keywords_bf, nombre_pages_bf):
    texte  = "%20".join(keyword.split())
    for i in range(1,last_page+1) : #On itère sur chaque page où il y a des articles
        driver.get('https://www.lemonde.fr/recherche/?search_keywords={}&start_at=01/01/2007&end_at=31/12/2022&search_sort=relevance_desc&page={}'.format(texte,i))
        liens_webscrapped = driver.find_elements(By.CLASS_NAME, "teaser__link") #On récupère les articles
    
        for lien in liens_webscrapped :
            liste_liens_bf.append(lien.get_attribute('href'))

In [ ]:
len(liste_liens_bf)

3874

In [ ]:
#Meme chose pour la liste des ministères

liste_liens_mf = []
error = []

for (keyword, last_page) in zip(liste_keywords_mf, nombre_pages_mf):
    texte  = "%20".join(keyword.lower().split())
    try :
        for i in range(1,last_page+1) : #On itère sur chaque page où il y a des articles
            driver.get('https://www.lemonde.fr/recherche/?search_keywords={}&start_at=01/01/2007&end_at=31/12/2022&search_sort=relevance_desc&page={}'.format(texte,i))
            liens_webscrapped = driver.find_elements(By.CLASS_NAME, "teaser__link") #On récupère les articles
    
            for lien in liens_webscrapped :
                if '-'.join(keyword.lower().split()) in lien.get_attribute('href'):
                        liste_liens_mf.append(lien.get_attribute('href'))
    
    except Exception as e: 
        error.append({lien: e})

In [ ]:
len(liste_liens_mf)

In [ ]:
#1er filtre : On commence par exclure les url sans lien avec notre analyse 

def theme(liste) :
    return pd.Series([x.split('/')[3] for x in liste])

In [ ]:
theme(liste_liens_bf).value_counts().head(20)

economie               1210
idees                   316
culture                 264
politique               250
societe                 187
europe                  185
la-crise-financiere     105
a-la-une                100
livres                   98
international            86
sport                    76
cinema                   75
m-le-mag                 44
crise-financiere         36
archives                 35
vous                     34
afrique                  34
argent                   33
m-styles                 30
planete                  27
dtype: int64

In [ ]:
theme(liste_liens_mf).value_counts().head(20)

politique                       1522
societe                          330
economie                         327
idees                            318
election-presidentielle-2022     229
election-presidentielle-2017     198
international                    197
les-decodeurs                    122
afrique                          102
planete                           77
emmanuel-macron                   53
europe                            47
a-la-une                          41
police-justice                    28
big-browser                       26
elections-legislatives-2022       21
pixels                            20
la-crise-financiere               19
elections-legislatives-2017       17
m-le-mag                          17
dtype: int64

In [ ]:
#on va garder uniquement les articles qui appartiennent aux catégories : economie/politique/crise-financière

liens_bf = [x for x in liste_liens_bf if x.split('/')[3] in ['economie', 'politique', 'la-crise-financiere', 'crise-financiere']]
liens_mf = [x for x in liste_liens_mf if x.split('/')[3] in ['economie', 'politique', 'la-crise-financiere', 'crise-financiere']]

In [ ]:
len(liens_bf)

1601

In [ ]:
len(liens_mf)

1873

In [ ]:
#On récupère les informations nécessaires de chaque article lié à la Banque de France :

list_of_articles = []
errors = []

for lien in liens_bf:
    try :
        response = requests.get(lien)
        soup = BeautifulSoup ( response.content , "html.parser")
        
        titre = soup.find('title').text   #scraper le titre
        
        # rajouter une condition sur les articles sans description :
        
        description = soup.find('p', attrs = {'class' : "article__desc"})   #scraper la description
        description = ['' if description == None else description.text]
        desc = description[0]
        
        article = soup.find_all('p', attrs = {'class' : 'article__paragraph'}) #scraper le contenu de l'article
        article_contenu = [x.text for x in article]
        texte  = " ".join(article_contenu)
        
        date = soup.find('meta', attrs= {'property' : "og:article:published_time"}) #scraper la date de publication
        date_publication = date.attrs['content'][:10]
        
        dict_articles = {'article_titre' : titre, 'article_desc' : desc, 'article_contenu' : texte, 
                         'article_date': date_publication,'article_lien' : lien}
        
        list_of_articles.append(dict_articles)
        time.sleep(2)
        
    except Exception as e: 
        errors.append({lien: e})

In [ ]:
len(list_of_articles)

1583

In [ ]:
#On récupère les informations nécessaires de chaque article lié aux Ministère des finances :

list_of_articles_mf = []
errors_mf = []

for lien in liens_mf:
    try :
        response = requests.get(lien)
        soup = BeautifulSoup ( response.content , "html.parser")
        
        titre = soup.find('title').text   #scraper le titre
        
        # rajouter une condition sur les articles sans description :
        
        description = soup.find('p', attrs = {'class' : "article__desc"})   #scraper la description
        description = ['' if description == None else description.text]
        desc = description[0]
        
        article = soup.find_all('p', attrs = {'class' : 'article__paragraph'}) #scraper le contenu de l'article
        article_contenu = [x.text for x in article]
        texte  = " ".join(article_contenu)
        
        date = soup.find('meta', attrs= {'property' : "og:article:published_time"}) #scraper la date de publication
        date_publication = date.attrs['content'][:10]
        
        dict_articles = {'article_titre' : titre, 'article_desc' : desc, 'article_contenu' : texte, 
                         'article_date': date_publication,'article_lien' : lien}
        
        list_of_articles_mf.append(dict_articles)
        time.sleep(2)
        
    except Exception as e: 
        errors_mf.append({lien: e})

In [ ]:
len(list_of_articles_mf)

1860

## 3/ Création de DataFrame et nettoyage des données :

In [ ]:
df_bf = pd.DataFrame(list_of_articles)
df_bf.head()

,article_titre,article_desc,article_contenu,article_date,article_lien
0,François Villeroy de Galhau : « La reprise se ...,L’économie française devrait rebondir de 1...,"En raison de la pandémie, l’économie tricolore...",2020-09-14,https://www.lemonde.fr/economie/article/2020/0...
1,François Villeroy de Galhau : « Il faut restau...,Dans sa traditionnelle « Lettre au préside...,"En 2019, alors que l’économie était au beau fi...",2020-07-09,https://www.lemonde.fr/economie/article/2020/0...
2,François Villeroy de Galhau : « Il faut ralen...,"Le gouverneur de la Banque de France, croi...",Alors que la croissance donne des signes de ra...,2018-06-20,https://www.lemonde.fr/economie/article/2018/0...
3,"« La France devrait éviter la récession », aff...","Dans un entretien au « Monde », François V...",Selon les projections macroéconomiques de la B...,2023-03-20,https://www.lemonde.fr/economie/article/2023/0...
4,Bataille à la Banque centrale européenne sur l...,S’il y a consensus pour relever le taux d’...,La Banque centrale européenne (BCE) s’apprête-...,2023-02-22,https://www.lemonde.fr/economie/article/2023/0...


In [ ]:
df_mf =  pd.DataFrame(list_of_articles_mf)
df_mf.head()

,article_titre,article_desc,article_contenu,article_date,article_lien
0,Bruno Le Maire reconnaît que le gouvernement «...,Le cabinet McKinsey est visé par une enquê...,"Le ministre de l’économie et des finances, Bru...",2022-11-27,https://www.lemonde.fr/politique/article/2022/...
1,"Inflation : Bruno Le Maire assure, étude à l’a...",Alors que l’inflation des produits aliment...,« Il n’y a pas eu de profiteurs de l’inflation...,2022-11-06,https://www.lemonde.fr/economie/article/2022/1...
2,Pénurie de carburant : le gouvernement réquisi...,"Emmanuel Macron a déclaré, lundi, vouloir ...",La grève se poursuit dans les raffineries et l...,2022-10-17,https://www.lemonde.fr/economie/article/2022/1...
3,"Budget, carburants… Bruno Le Maire, un ambitie...",Le ministre de l’économie a imposé ses arb...,Du théâtre ? Sans doute. Mais Bruno Le Maire e...,2022-10-19,https://www.lemonde.fr/politique/article/2022/...
4,Bruno Le Maire sermonne le Medef après une cri...,"Sur France Inter, le ministre de l’économi...",C’est « un peu fort de café ». Bruno Le Maire ...,2022-09-27,https://www.lemonde.fr/politique/article/2022/...


In [ ]:
df_mf['article_titre'].values[3].replace('\xa0', ' ')

'Budget, carburants… Bruno Le Maire, un ambitieux qui joue sa propre partition au sein du gouvernement'

In [ ]:
#2eme filtre : on ne garde pas les interviews avec les représentants de la BCE :

#On exclut donc les titres avec des citations 
#On exclut les descriptions avec le nom d'un des gouverneurs et le mot entretien
#On cherche d'autres critères pour exclure les interviews

def no_citation(df) :
    return [titre for titre in df['article_titre'] if '«' not in titre]

def no_entretien(df):
    return [desc for desc in df['article_desc'] if 'entretien' not in desc]

In [ ]:
#data_bf = df_bf[df_bf['article_titre'].isin(no_citation(df_bf)) | df_bf['article_titre'].isin(no_entretien(df_bf))]
data_mf = df_mf[df_mf['article_titre'].isin(no_entretien(df_mf)) | df_mf['article_titre'].isin(no_citation(df_mf))]

In [ ]:
data_bf.shape

(1509, 5)

In [ ]:
data_mf.shape

(1345, 5)

In [ ]:
#enlever les articles sans contenu:

data_bf = data_bf[data_bf['article_contenu'] != '']
data_mf = data_mf[data_mf['article_contenu'] != '']

In [ ]:
data_bf.shape, data_mf.shape

((1503, 5), (1317, 5))

In [ ]:
#merge description et texte :
data_bf['texte'] = data_bf['article_desc'] + data_bf['article_contenu']
data_mf['texte'] = data_mf['article_desc'] + data_mf['article_contenu']

In [ ]:
#Premier processing du texte :

def preprocess(txt):
    txt = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", txt)
    txt = txt.lower()
    txt = re.sub(r"\W"," ",txt)
    txt = re.sub(r"\d"," ",txt)
    txt = re.sub(r"\s+[a-z]\s+"," ",txt)
    txt = re.sub(r"\s+[a-z]$"," ",txt)
    txt = re.sub(r"^[a-z]\s+"," ",txt)
    txt = re.sub(r"\s+"," ",txt) 
    return txt

data_bf['Processed_texte'] = data_bf['texte'].apply(lambda txt: preprocess(txt))
data_mf['Processed_texte'] = data_mf['texte'].apply(lambda txt: preprocess(txt))

In [ ]:
data_bf.head()

,article_titre,article_desc,article_contenu,article_date,article_lien,texte,Processed_texte
4,Bataille à la Banque centrale européenne sur l...,S’il y a consensus pour relever le taux d’...,La Banque centrale européenne (BCE) s’apprête-...,2023-02-22,https://www.lemonde.fr/economie/article/2023/0...,S’il y a consensus pour relever le taux d’...,il a consensus pour relever le taux intérêt à...
5,"En France, un peu plus de croissance, un peu m...","Après la croissance de 2,6 % enregistrée e...","Le risque de récession, tant redouté pour 2023...",2023-02-07,https://www.lemonde.fr/economie/article/2023/0...,"Après la croissance de 2,6 % enregistrée e...",après la croissance de enregistrée en insee p...
6,"Après la chute de la plate-forme FTX, les cryp...","DécryptagesEffondrement des cours, faillites à...",Le bitcoin serait-il invulnérable ? Déclaré mo...,2023-01-29,https://www.lemonde.fr/economie/article/2023/0...,"DécryptagesEffondrement des cours, faillites à...",décryptageseffondrement des cours faillites à ...
7,Le taux du Livret A relevé de 2 % à 3 % au 1er...,"Le ministre de l’économie et des finances,...",Le taux appliqué au Livret A sera de 3 % à com...,2023-01-13,https://www.lemonde.fr/economie/article/2023/0...,"Le ministre de l’économie et des finances,...",le ministre de économie et des finances bruno...
8,La Banque de France anticipe un ralentissement...,Confrontée à « un choc extérieur majeur » ...,Après une croissance du produit intérieur brut...,2022-12-17,https://www.lemonde.fr/economie/article/2022/1...,Confrontée à « un choc extérieur majeur » ...,confrontée à un choc extérieur majeur avec la...


In [ ]:
#reset index :
data_bf = data_bf.reset_index(drop=True)
data_mf = data_mf.reset_index(drop=True)

In [ ]:
#calculer le total des mots :

data_bf['nb_mots'] = [len(x) - x.count(' ') for x in data_bf['Processed_texte']]
data_mf['nb_mots'] = [len(x) - x.count(' ') for x in data_mf['Processed_texte']]

In [ ]:
data_bf['nb_mots'].describe()

count     1503.000000
mean      2789.675981
std       1724.579121
min        183.000000
25%       1871.000000
50%       2294.000000
75%       3353.500000
max      18268.000000
Name: nb_mots, dtype: float64

In [ ]:
data_mf['nb_mots'].describe()

count     1345.000000
mean      2312.178439
std       1378.590700
min          0.000000
25%       1724.000000
50%       2163.000000
75%       2532.000000
max      18575.000000
Name: nb_mots, dtype: float64

In [ ]:
#On supprime les articles où on a pas de contenu :
data_mf = data_mf[data_mf['texte'].str.len() > 0]

In [ ]:
data_mf.describe()

,nb_mots
count,1338.000000
mean,2324.275037
std,1371.978787
min,49.000000
25%,1738.750000
50%,2163.500000
75%,2535.750000
max,18575.000000


In [ ]:
data_mf.shape

(1338, 8)

In [ ]:
#conserver la DataFrame en un fichier scv :
data_bf.to_csv('processing_data_bf.csv', index=False)

In [ ]:
data_mf.to_csv('processing_data_mf.csv', index=False)